# Extracting text from PDF files

In [1]:
# To read the PDF
import PyPDF2
# To analyze the PDF layout and extract text
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# To extract text from tables in PDF
import pdfplumber
# To extract the images from the PDFs
from PIL import Image
from pdf2image import convert_from_path
# To perform OCR to extract text from images 
import pytesseract 
# To remove the additional created files
import os

In [ ]:
# from tika import parser # pip install tika

# raw = parser.from_file('./files/Toyota.pdf')
# print(raw['content'])

In [3]:
from pypdf import PdfReader

reader = PdfReader('./files/SPD-SX_PRO.pdf')
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"

In [ ]:
# print(text)

In [8]:
import re

def remove_index_content(text):
    # Define regular expressions for common index patterns
    index_patterns = [
        r'\.\.\.\s*[pP]\.\s*\d+',  # Matches entries like "... P. 500"
        r'\b\d+\s*[-\.]?\s*[A-Za-z]+\s*\.{3}\s*\d+',  # Matches entries like "32. Before driving... 32"
        r'\b[A-Za-z]+\s*\.{3}\s*\d+',  # Matches entries like "Transmission ... 227"
        r'^\s*\*\d+',  # Matches entries like "*2"
        r'^\d+$',  # Matches entries like "1"
        r'^\d+\s*[-\.]?\s*\d+\s*\.\s*[A-Za-z]+',  # Matches entries like "33 1-1. For safe use"
        r'^Table \d+\.\d+\s+',  # Matches entries like "Table 7.7  Difference between metronomic and actual onsets of subdivisions"
        r'^Figure \d+\.\d+\s+',  # Matches entries like "Figure 2.1  Bloco afro Olodum parading in Salvador and Olodum’s logo exhibiting"
        # Add more patterns as needed
    ]

    # Combine patterns into a single regular expression
    index_regex = re.compile('|'.join(index_patterns), re.MULTILINE)

    # Use the regular expression to remove index-like content
    cleaned_text = re.sub(index_regex, '', text)

    # Remove lines with 3 or more dots or spaces
    cleaned_text = '\n'.join(line for line in cleaned_text.split('\n') if line.count('.') < 3 and line.count(' . ') < 3) 

    cleaned_text = cleaned_text.replace('■', ' ').replace('●', ' ')

    return cleaned_text



In [9]:
# import re

# def clean(text):
    
#     # removing paragraph numbers
#     text = re.sub('[0-9]+.\t','',str(text))
#     # removing new line characters
#     text = re.sub('\n ','',str(text))
#     text = re.sub('\n',' ',str(text))
#     # removing apostrophes
#     text = re.sub("'s",'',str(text))
#     # removing hyphens
#     text = re.sub("-",' ',str(text))
#     text = re.sub("— ",'',str(text))
#     # removing quotation marks
#     text = re.sub('\"','',str(text))
#     # removing salutations
#     text = re.sub("Mr\.",'Mr',str(text))
#     text = re.sub("Mrs\.",'Mrs',str(text))
#     # removing any reference to outside text
#     text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    
#     return text

In [ ]:
# input_str = input_str.lower()
# input_str = input_str.strip()


In [ ]:
fred = remove_index_content(text)
# resultFred = clean(fred)
print(fred)

In [6]:
with open('./files/SPD-SX_PRO.txt', 'w') as file:
    # Write text to the file
    file.write(fred)

In [2]:
import os
from io import StringIO
import re
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

In [3]:
def pdf_to_text(path):
    '''Extract text from pdf documents
    '''

    manager = PDFResourceManager()
    retstr = StringIO()
    layout = LAParams(all_texts=False, detect_vertical=True)
    device = TextConverter(manager, retstr, laparams=layout)
    interpreter = PDFPageInterpreter(manager, device)
    with open(path, 'rb') as filepath:
        for page in PDFPage.get_pages(filepath, check_extractable=True):
            interpreter.process_page(page)
    text = retstr.getvalue()
    device.close()
    retstr.close()
    return text

In [5]:
text_from_pdf = pdf_to_text('./files/SPD-SX_PRO.pdf')

In [ ]:
print(text_from_pdf)

In [11]:
with open('./files/SPD-SX_PRO.txt', 'w') as file:
    # Write text to the file
    file.write(text_from_pdf)